In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

class SWARA_MCDA:
    def __init__(self, expert_judgments):
        self.expert_judgments = expert_judgments

    def calculate_weights(self):
        s_values = np.array(self.expert_judgments)
        k_values = np.zeros_like(s_values, dtype=float)
        q_values = np.zeros_like(s_values, dtype=float)

        for i in range(len(s_values)):
            if i == 0:
                k_values[i] = 1
            else:
                tradeoff = s_values[i] / s_values[i - 1] if s_values[i - 1] != 0 else 1
                k_values[i] = 1 + s_values[i] * tradeoff

        q_values[0] = 1
        for i in range(1, len(k_values)):
            q_values[i] = q_values[i - 1] * k_values[i]

        #Weights normalized to make sure they add up to 1
        weights = 1 / q_values
        w_norm = weights / weights.sum()
        return w_norm

In [ ]:
def compute_dcg(scores, k=None):
    scores = np.array(scores[:k]) if k else np.array(scores)
    return np.sum((2 ** scores - 1) / np.log2(np.arange(2, len(scores) + 2)))

def group_queries(data, query_col):
    queries = {}
    for idx, query_id in enumerate(data[:, query_col]):
        queries.setdefault(query_id, []).append(idx)
    return queries
def compute_pairs(scores):
    return [(i, j) for i in range(len(scores)) for j in range(len(scores)) if scores[i] > scores[j]]

def compute_ndcg(scores, k=None):
    ideal_scores = sorted(scores, reverse=True)
    idcg = compute_dcg(ideal_scores, k)
    if idcg == 0:
        return 0
    return compute_dcg(scores, k) / idcg

def compute_lambda(true_scores, pred_scores, pairs, idcg):
    if idcg == 0:
        return np.zeros(len(pred_scores)), np.zeros(len(pred_scores))

    lambdas = np.zeros(len(pred_scores))
    weights = np.zeros(len(pred_scores))

    for i, j in pairs:
        delta_dcg = abs(
            (2 ** true_scores[i] - 2 ** true_scores[j]) / np.log2(j + 2)
            - (2 ** true_scores[j] - 2 ** true_scores[i]) / np.log2(i + 2)
        ) / idcg
        rho = 1 / (1 + np.exp(pred_scores[i] - pred_scores[j]))
        lambda_update = delta_dcg * rho
        lambdas[i] += lambda_update
        lambdas[j] -= lambda_update
        weights[i] += rho * (1 - rho) * delta_dcg
        weights[j] += rho * (1 - rho) * delta_dcg

    return lambdas, weights

class LambdaMART:
    def __init__(self, num_trees: int = 5, max_depth: int = 10, learning_rate: float = 0.1,
                 min_samples_split: int = 10, min_samples_leaf: int = 5, l2_reg: float = 0.01):
        self.num_trees = num_trees
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.l2_reg = l2_reg
        self.trees = []
        self.fitted_scores = None

    def fit(self, x: np.ndarray, y: np.ndarray):
        queries = group_queries(x, 1)
        pred_scores = np.zeros(len(y))

        idcgs = {q: compute_dcg(sorted(y[indices], reverse=True)) for q, indices in queries.items()}

        for _ in range(self.num_trees):
            lambdas = np.zeros(len(y))

            for q, indices in queries.items():
                true_scores = y[indices]
                pred_scores_q = pred_scores[indices]
                pairs = compute_pairs(true_scores)
                idcg = idcgs[q]

                lambda_vals, _ = compute_lambda(true_scores, pred_scores_q, pairs, idcg)
                lambdas[indices] += lambda_vals

            tree = DecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf
            )
            tree.fit(x[:, 2:], lambdas)
            self.trees.append(tree)

            pred_scores += self.learning_rate * tree.predict(x[:, 2:]) - self.l2_reg * pred_scores
        self.fitted_scores = pred_scores
        return np.argsort(-pred_scores), pred_scores

    def predict(self, x: np.ndarray):
      # Ensure the model is fitted
      if self.fitted_scores is None:
        raise ValueError("The model has not been fitted yet. Call fit before predict.")


      new_scores = np.zeros(x.shape[0])
      for tree in self.trees:
        new_scores += self.learning_rate * tree.predict(x[:, 2:])


      all_scores = np.concatenate([self.fitted_scores, new_scores])
      ranked_indices = np.argsort(-all_scores)
      rank_map = {idx: rank + 1 for rank, idx in enumerate(ranked_indices)}


      new_input_start = len(self.fitted_scores)
      new_ranks = [rank_map[idx] for idx in range(new_input_start, new_input_start + len(new_scores))]

      return new_ranks, new_scores

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def load_data():
    df = pd.read_csv("data2.csv").dropna()

    df = df.sample(frac=1, random_state=42)

    if 'minmax' not in df.columns:
        raise ValueError("The 'minmax' column is not present in the CSV file.")


    minmax_values = df['minmax']

    relevant_columns = ['Active Personnel', 'Aircraft Carriers', 'Available Manpower',
                        'Defense Budget', 'Fit-for-Service', 'Labor Force', 'Oil Consumption',
                        'Total Population', 'Total Aircraft Strength']

    for col in relevant_columns:
        if col in df.columns:
            df[col] = df[col] * minmax_values

    country_names = df['Country']
    X = df[relevant_columns]

    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(X)

    return df, country_names, X_normalized, scaler

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

def plot_ranking(dfY_sort, ranked_countries, relevance_scores):
    """
    Plots a bar chart for ranked countries with relevance scores.

    Parameters:
        dfY_sort (pd.DataFrame): DataFrame with sorted data.
        ranked_countries (list): List of ranked country abbreviations/names.
        relevance_scores (list): List of relevance scores.
    """

    sorted_indices = sorted(range(len(relevance_scores)), key=lambda k: relevance_scores[k], reverse=True)
    ranks = ['R' + str(sorted_indices.index(i) + 1) for i in range(len(relevance_scores))]


    figure = plt.figure(figsize=(len(ranked_countries) * 1.2, 6))


    plt.bar(range(1, len(relevance_scores) + 1), relevance_scores, color='grey', width=0.5)

    plt.xticks(range(1, len(relevance_scores) + 1), ranked_countries, fontsize=14, rotation=45, ha='right')


    plt.xlabel('Države svijeta', fontsize=14, labelpad=20)
    plt.ylabel('Relevantnost', fontsize=14)
    plt.yticks(fontsize=14)

    plt.axhline(0, color='black', linewidth=1)


    for i, (score, rank) in enumerate(zip(relevance_scores, ranks)):
        vertical_offset = 0.0001 if score >= 0 else -0.004
        plt.text(i + 1, score + vertical_offset, rank,
                 size=14, ha='center', va='bottom' if score >= 0 else 'top', color='black')


    plt.gca().yaxis.set_major_locator(MultipleLocator(0.2))
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.2f'))


    plt.margins(0.05, 0.05)
    plt.tight_layout()
    plt.show()


In [ ]:
def main():
    data, country_names, X_normalized, scaler = load_data()

    features = ['Active Personnel', 'Aircraft Carriers', 'Available Manpower',
                'Defense Budget', 'Fit-for-Service', 'Labor Force', 'Oil Consumption',
                'Total Population','Total Aircraft Strength']

    X = data[features]
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X)

    # Generate expert judgments for SWARA
    expert_judgments = np.random.dirichlet(np.ones(len(features)))
    #expert_judgments=[0.03753326, 0.02667568, 0.10663758, 0.17123656, 0.02837184, 0.12319237, 0.06684213, 0.2574416,  0.18206899] TEST FOR WEIGHTS I HAD ALREADY COMPARISSON
    print(f'Expert judgments: {expert_judgments}')
    print(f'Sum of expert judgments: {np.sum(expert_judgments)}')  # Confirm sum is 1


    swara = SWARA_MCDA(expert_judgments)
    weights = swara.calculate_weights()
    print('Weights', weights)


    relevance_scores = np.dot(X_normalized, weights)


    lambda_mart = LambdaMART()
    ranked_indices, pred_scores=lambda_mart.fit(X_normalized, relevance_scores)
    ranked_countries = data['Country'].iloc[ranked_indices].reset_index(drop=True)

    dfY_sort = pd.DataFrame({'Rank': ranked_indices, 'Score': relevance_scores[ranked_indices]})


    plot_ranking(dfY_sort, ranked_countries, relevance_scores)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def load_data():
    df = pd.read_csv("C:/Users/azhar/Desktop/Stl/global firepower 2022 wide.csv").dropna()


    df = df.sample(frac=1, random_state=42)

    if 'minmax' not in df.columns:
        raise ValueError("The 'minmax' column is not present in the CSV file.")


    minmax_values = df['minmax']
    relevant_columns = ['Active Personnel', 'Aircraft Carriers', 'Available Manpower',
                        'Defense Budget', 'Fit-for-Service', 'Labor Force', 'Oil Consumption',
                        'Total Population', 'Total Aircraft Strength']

    for col in relevant_columns:
        if col in df.columns:
            df[col] = df[col] * minmax_values


    country_names = df['country']
    country_continent = df['continent'].unique()

    X = df[relevant_columns]


    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X)

    return df, country_names, X_normalized, scaler, country_continent


In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Main App
def app():
    # Streamlit App Title
    st.title("🌍 Military Ranking")
    st.markdown("Rank existing countries or add a new fake country to see how it compares!")

    # User Input for Expert Judgments
    st.write("### ✍️ Input Expert Judgments")
    criteria_names = ['Active Personnel', 'Aircraft Carriers', 'Available Manpower', 'Defense Budget',
                      'Fit-for-Service', 'Labor Force', 'Oil Consumption',  'Total Population','Total Aircraft Strength']
    expert_judgments = []

    # Create 4 columns for input fields
    columns = st.columns(3)
    for i, criterion in enumerate(criteria_names):
        column = columns[i % 3]
        value = column.number_input(f"{criterion}:", min_value=0.0, max_value=1.0, value=0.0, step=0.01, key=criterion)
        expert_judgments.append(value)

    # Display the sum of expert judgments
    sum_weights = sum(expert_judgments)
    st.write(f"### Total of Expert Judgments: {sum_weights:.2f}")

    # Check if the sum of expert judgments equals 1 and enable ranking only if valid
    if not np.isclose(sum_weights, 1.0):
        st.warning("The total of expert judgments must equal 1. Please adjust the values.")
        return  # Stop further execution if sum is not 1

    # Calculate Weights using SWARA
    swara = SWARA_MCDA(expert_judgments)
    weights = swara.calculate_weights()

    # Display the SWARA weights
    columns = st.columns(3)
    for i, (criterion, weight) in enumerate(zip(criteria_names, weights)):
        column = columns[i % 3]
        column.metric(label=f"{criterion}:", value=f"{weight:.2f}")

    # Load and Process Data
    try:
        data, country_names, X_normalized, scaler, continent = load_data()
    except FileNotFoundError:
        st.error("Data file not found. Please ensure 'global firepower 2022 wide.csv' is in the correct directory.")
        return


    # Calculate relevance scores
    relevance_scores = np.dot(X_normalized, weights)

    # Train LambdaMART ranking model
    lambda_mart = LambdaMART()
    # Get ranked indices from LambdaMART
    ranked_indices, pred_scores =lambda_mart.fit(X_normalized, relevance_scores)

    # Display ranking results
    st.write("### Ranking Results:")
    st.write("Ranking countries based on the expert judgments...")

    # Retrieve ranked country names
    ranked_countries = data['country'].iloc[ranked_indices].reset_index(drop=True)

    # Dropdown menu for country selection
    selected_country = st.selectbox("Select a country:", ranked_countries)

    # Display the rank of the selected country
    if selected_country:
      rank = ranked_countries[ranked_countries == selected_country].index[0] + 1
      st.write(f"The rank of **{selected_country}** is: **{rank}**")    # Section to Add and Rank a New Fake Country
    st.write("### 🏴 Add and Rank a New Fake Country")

    # Input fields for the fake country data
    st.write("Enter the data for your fake country:")
    fake_country_data = {}

    # Create 3 columns for the input fields
    columns = st.columns(3)

    # Display 3 input fields per row
    fake_country_data['Active Personnel'] = columns[0].number_input("Active Personnel", min_value=0, step=1)
    fake_country_data['Aircraft Carriers'] = columns[1].number_input("Aircraft Carriers", min_value=0, step=1)
    fake_country_data['Available Manpower'] = columns[2].number_input("Available Manpower", min_value=0, step=1)

    fake_country_data['Defense Budget'] = columns[0].number_input("Defense Budget (in mil USD)", min_value=0, step=1)*1000000
    fake_country_data['Fit-for-Service'] = columns[1].number_input("Fit-for-Service", min_value=0, step=1)
    fake_country_data['Labor Force'] = columns[2].number_input("Labor Force", min_value=0, step=1)

    fake_country_data['Oil Consumption'] = columns[0].number_input("Oil Consumption", min_value=0, step=1)
    fake_country_data['Total Population'] = columns[1].number_input( 'Total Population', min_value=0, step=1)
    fake_country_data['Total Aircraft Strength'] = columns[2].number_input('Total Aircraft Strength', min_value=0, step=1)
    fake_country_df = pd.DataFrame([fake_country_data])

    # Button to add the fake country and rank it
    if st.button("Rank Fake Country"):

        # Normalize the fake country data using the existing scaler
        fake_country_normalized = scaler.transform(fake_country_df[criteria_names])

        # Re-rank countries including the fake country
        new_rank,_ = lambda_mart.predict(fake_country_normalized)

        # Display ranking results including the fake country
        st.write("### Updated Ranking with Fake Country:")
        for i, country in enumerate(new_rank, start=1):
          st.write(f"The rank of your **Fake Country** is: {country}")

        # Top 10 Ranked Countries
        st.write("### 🏆 Top 10 Highest Ranked Countries")

        # Display the top 10 ranked countries
        top_10_countries = ranked_countries.head(10)  # Select the top 10
        top_10_scores = pred_scores[ranked_indices[:10]]  # Get their predicted scores

        # Create a DataFrame for display
        top_10_df = pd.DataFrame({ "Rank": range(1, 11),"Country": top_10_countries,"Relevance score": top_10_scores})

        # Reset index and drop it
        top_10_df.reset_index(drop=True, inplace=True)

        # Display the DataFrame without an extra index
        st.table(top_10_df.set_index("Rank"))



In [ ]:
if __name__ == "__main__":
  #Change from main() to app() when you want to use streamlit
  #Use main for the MCDA comparison
    #main()
    app()
